In [19]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd

class Candle:
    def __init__(self, open, high, low, close):
        self.open = open
        self.high = high
        self.low = low
        self.close = close
        self._validate_candle()

    def __repr__(self):
        return f"Candle({self.open}, {self.high}, {self.low}, {self.close})"

    def __str__(self):
        return f"O: {self.open}, H: {self.high}, L: {self.low}, C: {self.close}"

    @property
    def open(self):
        return self._open

    @property
    def high(self):
        return self._high

    @property
    def low(self):
        return self._low

    @property
    def close(self):
        return self._close

    @open.setter
    def open(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("open must be int or float not {}".format(type(value)))
        if value < 0:
            raise ValueError("open must be positive")
        self._open = value

    @high.setter
    def high(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("high must be int or float not {}".format(type(value)))
        if value < 0:
            raise ValueError("high must be positive")
        self._high = value

    @low.setter
    def low(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("low must be int or float not {}".format(type(value)))
        if value < 0:
            raise ValueError("Low must be positive")
        self._low = value

    @close.setter
    def close(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("close must be int or float not {}".format(type(value)))
        if value < 0:
            raise ValueError("close must be positive")
        self._close = value

    def _validate_candle(self):
        if self.open > self.high:
            raise ValueError("open cannot be greater than high")
        if self.open < self.low:
            raise ValueError("open cannot be less than low")
        if self.close > self.high:
            raise ValueError("close cannot be greater than high")
        if self.close < self.low:
            raise ValueError("close cannot be less than low")


class CandleFrame:
    def __init__(self, open, high, low, close):
        open, high, low, close = self._validate_input(open, high, low, close)
        self.candles = [Candle(o, h, l, c) for o, h, l, c in zip(open, high, low, close)]
        self.df = pd.DataFrame({"open": open, "high": high, "low": low, "close": close})

    def __repr__(self):
        return f"CandleFrame({self.df})"

    def __str__(self):
        return f"{self.df}"

    def __len__(self):
        return len(self.candles)

    def __getitem__(self, index):
        return self.candles[index]

    def __iter__(self):
        return iter(self.candles)

    def __reversed__(self):
        return reversed(self.candles)

    @staticmethod
    def _validate_input(open, high, low, close):
        if not isinstance(open, list) and not isinstance(open, np.ndarray) and not isinstance(open, pd.DataFrame):
            raise TypeError("open must be list, np.ndarry, or pd.DataFrame not {}".format(type(open)))
        if not isinstance(high, list) and not isinstance(high, np.ndarray) and not isinstance(high, pd.DataFrame):
            raise TypeError("high must be list, np.ndarry, or pd.DataFrame not {}".format(type(high)))
        if not isinstance(low, list) and not isinstance(low, np.ndarray) and not isinstance(low, pd.DataFrame):
            raise TypeError("low must be list, np.ndarry, or pd.DataFrame not {}".format(type(low)))
        if not isinstance(close, list) and not isinstance(close, np.ndarray) and not isinstance(close, pd.DataFrame):
            raise TypeError("close must be list, np.ndarry, or pd.DataFrame not {}".format(type(close)))
        open = list(open)
        high = list(high)
        low = list(low)
        close = list(close)
        if len(open) != len(high) or len(open) != len(low) or len(open) != len(close):
            raise ValueError("open, high, low, and close must be the same length")
        if not all(isinstance(x, (int, float)) for x in open):
            raise TypeError("open must be list of int or float not {}".format(type(open)))
        if not all(isinstance(x, (int, float)) for x in high):
            raise TypeError("high must be list of int or float not {}".format(type(high)))
        if not all(isinstance(x, (int, float)) for x in low):
            raise TypeError("low must be list of int or float not {}".format(type(low)))
        if not all(isinstance(x, (int, float)) for x in close):
            raise TypeError("close must be list of int or float not {}".format(type(close)))
        return open, high, low, close

In [32]:
# make a candle frame
open = [1, 2, 3, 4, 5]
high = [2, 3, 4, 5, 6]
low = [0, 1, 2, 3, 4]
close = [1.5, 2.5, 3.5, 4.5, 5.5]
candle_frame = CandleFrame(open, high, low, close)

In [24]:
for candle in candle_frame:
    print(candle)

O: 1, H: 2, L: 0, C: 1.5
O: 2, H: 3, L: 1, C: 2.5
O: 3, H: 4, L: 2, C: 3.5
O: 4, H: 5, L: 3, C: 4.5
O: 5, H: 6, L: 4, C: 5.5
